<a href="https://colab.research.google.com/github/MTajuddin4u/Assigment-04/blob/main/password_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import random
import string
import secrets
import pyperclip  # For copy to clipboard functionality

def generate_password(length, use_upper, use_lower, use_digits, use_special, avoid_similar, avoid_ambiguous):
    character_sets = []

    if use_upper:
        upper_chars = string.ascii_uppercase
        if avoid_similar:
            upper_chars = upper_chars.replace('I', '').replace('O', '')
        character_sets.append(upper_chars)

    if use_lower:
        lower_chars = string.ascii_lowercase
        if avoid_similar:
            lower_chars = lower_chars.replace('l', '')
        character_sets.append(lower_chars)

    if use_digits:
        digit_chars = string.digits
        if avoid_ambiguous:
            digit_chars = digit_chars.replace('0', '').replace('1', '')
        character_sets.append(digit_chars)

    if use_special:
        special_chars = "!@#$%^&*()_-+=<>?/[]{}|"
        if avoid_ambiguous:
            special_chars = special_chars.replace('<', '').replace('>', '').replace('(', '').replace(')', '')
            special_chars = special_chars.replace('[', '').replace(']', '').replace('{', '').replace('}', '')
            special_chars = special_chars.replace('|', '').replace('\\', '').replace('/', '').replace('`', '')
        character_sets.append(special_chars)

    if not character_sets:
        return "Please select at least one character type"

    # Ensure at least one character from each selected set
    password = []
    for charset in character_sets:
        password.append(secrets.choice(charset))

    # Fill the rest of the password
    all_chars = ''.join(character_sets)
    password.extend(secrets.choice(all_chars) for _ in range(length - len(password)))

    # Shuffle to mix the mandatory characters
    random.shuffle(password)

    return ''.join(password)

def password_strength(password):
    if len(password) < 8:
        return "Very Weak", 1
    elif len(password) < 12:
        return "Weak", 2
    elif len(password) < 16:
        return "Moderate", 3
    else:
        has_upper = any(c.isupper() for c in password)
        has_lower = any(c.islower() for c in password)
        has_digit = any(c.isdigit() for c in password)
        has_special = any(not c.isalnum() for c in password)

        complexity = sum([has_upper, has_lower, has_digit, has_special])

        if complexity == 4:
            return "Very Strong", 5
        elif complexity >= 3:
            return "Strong", 4
        else:
            return "Moderate", 3

def main():
    st.title("🔐 Secure Password Generator")

    with st.sidebar:
        st.header("Password Tips")
        st.write("- Use at least 12 characters")
        st.write("- Include uppercase, lowercase, numbers, and symbols")
        st.write("- Avoid common words and personal info")
        st.write("- Don't reuse passwords across sites")

    col1, col2 = st.columns(2)

    with col1:
        length = st.slider("Password Length", 4, 64, 12)
        use_upper = st.checkbox("Uppercase Letters (A-Z)", True)
        use_lower = st.checkbox("Lowercase Letters (a-z)", True)

    with col2:
        use_digits = st.checkbox("Digits (0-9)", True)
        use_special = st.checkbox("Special Characters (!@#...)", True)

    st.subheader("Advanced Options")
    advanced_col1, advanced_col2 = st.columns(2)

    with advanced_col1:
        avoid_similar = st.checkbox("Avoid similar characters (I, l, O, 0)")

    with advanced_col2:
        avoid_ambiguous = st.checkbox("Avoid ambiguous characters ({}[]()/\\`~)")

    if st.button("Generate Password"):
        password = generate_password(
            length=length,
            use_upper=use_upper,
            use_lower=use_lower,
            use_digits=use_digits,
            use_special=use_special,
            avoid_similar=avoid_similar,
            avoid_ambiguous=avoid_ambiguous
        )

        if password.startswith("Please select"):
            st.warning(password)
        else:
            strength, level = password_strength(password)

            # Display password in a nice box
            st.subheader("Generated Password")
            st.code(password, language="text")

            # Strength indicator
            st.write(f"Strength: {strength}")
            st.progress(level/5)

            # Copy to clipboard button
            if st.button("Copy to Clipboard"):
                pyperclip.copy(password)
                st.success("Password copied to clipboard!")

            # Password analysis
            with st.expander("Password Analysis"):
                st.write(f"Length: {len(password)} characters")
                has_upper = any(c.isupper() for c in password)
                has_lower = any(c.islower() for c in password)
                has_digit = any(c.isdigit() for c in password)
                has_special = any(not c.isalnum() for c in password)

                st.write("Contains:")
                cols = st.columns(4)
                cols[0].metric("Uppercase", "✓" if has_upper else "✗")
                cols[1].metric("Lowercase", "✓" if has_lower else "✗")
                cols[2].metric("Digits", "✓" if has_digit else "✗")
                cols[3].metric("Special", "✓" if has_special else "✗")

if __name__ == "__main__":
    main()